In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# !pip install modin

## 数据预处理

In [3]:
import modin.pandas as pd
from fastai.tabular import *
import time

2019-05-08 16:56:22,704	WARNING worker.py:1337 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-05-08 16:56:22,705	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-08_16-56-22_32567/logs.
2019-05-08 16:56:22,810	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:10199 to respond...
2019-05-08 16:56:22,919	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:23527 to respond...
2019-05-08 16:56:22,921	INFO services.py:804 -- Starting Redis shard with 10.0 GB max memory.
2019-05-08 16:56:22,929	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-08_16-56-22_32567/logs.
2019-05-08 16:56:22,931	WARNING services.py:1304 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 33711022080 bytes available. This may slow dow

In [4]:
root = Path('../test_A')
root
root.ls()

PosixPath('../test_A')

[PosixPath('../test_A/ad_static_feature.out'),
 PosixPath('../test_A/ad_operation.dat'),
 PosixPath('../test_A/.ipynb_checkpoints'),
 PosixPath('../test_A/imps_log'),
 PosixPath('../test_A/test_sample.dat'),
 PosixPath('../test_A/user')]

In [5]:
user = root/'user'
user.ls()
imps_log = root/'imps_log'
imps_log.ls()

[PosixPath('../test_A/user/user_data')]

[PosixPath('../test_A/imps_log/totalExposureLog.out')]

- 历史曝光日志数据文件

In [6]:
col_names = ['广告请求id', '广告请求时间', '广告位id', '用户id', '曝光广告id', '曝光广告素材尺寸', '曝光广告出价bid',
             '曝光广告pctr', '曝光广告quality_ecpm', '曝光广告totalEcpm']

In [7]:
imps_log = pd.read_csv(imps_log/'totalExposureLog.out', sep='\t', header=None, names=col_names, iterator=True)
imps_log.get_chunk(5)

,广告请求id,广告请求时间,广告位id,用户id,曝光广告id,曝光广告素材尺寸,曝光广告出价bid,曝光广告pctr,曝光广告quality_ecpm,曝光广告totalEcpm
0,53991770,1550409746,94,1160618,451525,50,46,47.217,944.34,3122.340
1,25942318,1550370892,79,203814,214797,64,10,49.094,981.88,1471.880
2,66156247,1550416600,18,808543,92253,40,96,3.824,76.48,443.584
3,5935886,1550365898,198,7270,160082,64,85,6.123,122.46,642.915
4,11624425,1550361159,168,852707,253902,64,60,4.329,86.58,346.320


In [8]:
p1 = imps_log.get_chunk(10)

- 1970 纪元后经过的浮点秒数

In [9]:
p1

,广告请求id,广告请求时间,广告位id,用户id,曝光广告id,曝光广告素材尺寸,曝光广告出价bid,曝光广告pctr,曝光广告quality_ecpm,曝光广告totalEcpm
5,87085212,1550406108,209,794015,126362,64,236,13.914,278.28,3561.984
6,13220086,1550412325,320,228155,307878,64,107,7.538,150.76,957.326
7,86036534,1550366402,39,1089376,324657,44,31,18.965,379.30,966.300
8,14581426,1550333073,297,601634,427309,64,42,21.066,421.32,1306.092
9,35294027,1550379093,297,1373800,608799,64,12,7.194,143.88,230.208
10,86704947,1550379451,56,883084,165604,36,93,2.581,51.62,291.620
11,31916513,1550377336,1,570901,401654,44,70,16.057,321.14,1444.140
12,83026881,1550369695,18,1168311,509332,64,34,8.891,177.82,479.820
13,80328122,1550386488,215,1255360,17155,64,128,15.200,304.00,2249.600
14,34666676,1550409564,198,417201,359175,40,30,3.388,67.76,169.400


In [10]:
p1['广告请求时间'] = pd.to_datetime(p1['广告请求时间'], unit='s')

In [11]:
p1

,广告请求id,广告请求时间,广告位id,用户id,曝光广告id,曝光广告素材尺寸,曝光广告出价bid,曝光广告pctr,曝光广告quality_ecpm,曝光广告totalEcpm
5,87085212,2019-02-17 12:21:48,209,794015,126362,64,236,13.914,278.28,3561.984
6,13220086,2019-02-17 14:05:25,320,228155,307878,64,107,7.538,150.76,957.326
7,86036534,2019-02-17 01:20:02,39,1089376,324657,44,31,18.965,379.30,966.300
8,14581426,2019-02-16 16:04:33,297,601634,427309,64,42,21.066,421.32,1306.092
9,35294027,2019-02-17 04:51:33,297,1373800,608799,64,12,7.194,143.88,230.208
10,86704947,2019-02-17 04:57:31,56,883084,165604,36,93,2.581,51.62,291.620
11,31916513,2019-02-17 04:22:16,1,570901,401654,44,70,16.057,321.14,1444.140
12,83026881,2019-02-17 02:14:55,18,1168311,509332,64,34,8.891,177.82,479.820
13,80328122,2019-02-17 06:54:48,215,1255360,17155,64,128,15.200,304.00,2249.600
14,34666676,2019-02-17 13:19:24,198,417201,359175,40,30,3.388,67.76,169.400


In [12]:
p1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 5 to 14
Data columns (total 10 columns):
广告请求id              10 non-null int64
广告请求时间              10 non-null datetime64[ns]
广告位id               10 non-null int64
用户id                10 non-null int64
曝光广告id              10 non-null int64
曝光广告素材尺寸            10 non-null int64
曝光广告出价bid           10 non-null int64
曝光广告pctr            10 non-null float64
曝光广告quality_ecpm    10 non-null float64
曝光广告totalEcpm       10 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(6)
memory usage: 884.0 bytes


In [13]:
def isVaildDate(date_str):
        try:
            if ":" in date_str:
                time.strptime(date_str, "%Y-%m-%d %H:%M:%S")
            else:
                time.strptime(date_str, "%Y-%m-%d")
            return True
        except:
            return False 

In [14]:
isVaildDate('2019-02-31 12:21:48')

False

In [15]:
isVaildDate(str(p1.loc[10, '广告请求时间']))

True

- 用户特征属性文件

In [16]:
col_names = ['用户id', 'Age', 'Gender', '地域', '婚恋状态', '学历', '消费能力',
             '设备', '工作状态', '连接类型', '行为兴趣']

In [17]:
user_data = pd.read_csv(user/'user_data', sep='\t', header=None, names=col_names, iterator=True)
user_data.get_chunk(5)

,用户id,Age,Gender,地域,婚恋状态,学历,消费能力,设备,工作状态,连接类型,行为兴趣
0,624218,753,2,"523,7589,12527,9019,14605,13489,302,6222,14214...",0,6,1,2,0,4,"32612,34406,3413,33059,4275,4667,4561,34051,42..."
1,271475,601,3,"9019,9032,663,12527,14532,2416,2599,1987,9687,...",13,7,2,2,0,2,"4990,4992,3413,33061,4115,4876,2393,34167,3441..."
2,44439,819,3,"12527,14188,9841,9019,14605,6222,9962,6643,172...",13,7,3,2,4,4,"3099,4403,34414,431,34415,5108,31686,3240,3229..."
3,863717,1,2,"12527,3435,2398,11069,1292,9019,14605,14348,62...",6,5,3,2,0,2,"34293,845,3413,3099,33061,33059,4120,4410,4415..."
4,1305469,714,3,"8928,9019,1540,719,12527,6051,10545,12118,7107...",13,5,3,2,0,2,"32612,34289,34406,3413,4275,4410,4561,34051,34..."


- 广告静态数据

In [18]:
col_names = ['广告id', '创建时间', '广告账户id', '商品id', '商品类型', '广告行业id', '素材尺寸']

In [19]:
ad_static_feature = pd.read_csv(root/'ad_static_feature.out', sep='\t', header=None, names=col_names, low_memory=False, iterator=True)
ad_static_feature.get_chunk(5)

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
0,106452,1529958950,22226,16088,13,225,NaN
1,233649,1538221936,25681,7356,13,136,1.0
2,547531,1550731020,20696,-1,1,186,40.0
3,707841,1551857857,3968,-1,3,186,40.0
4,457009,1550439402,23614,7447,13,172,NaN


- 广告操作数据

In [20]:
col_names = ['广告id', '创建/修改时间', '操作类型', '修改字段', '操作后的字段值', '人群定向']

In [21]:
ad_operation = pd.read_csv(root/'ad_operation.dat', sep='\t', header=None, names=col_names, iterator=True)
ad_operation.get_chunk(5)

,广告id,创建/修改时间,操作类型,修改字段,操作后的字段值,人群定向
0,593323,0,2,2,90,NaN
1,593323,0,2,3,all,NaN
2,593323,0,2,4,"281474976710655,281474976710655,28147497671065...",NaN
3,593323,20190217000000,1,1,0,NaN
4,593323,20190218233855,1,2,90,NaN


- 测试数据

In [22]:
col_names = ['样本id', '广告id', '创建时间', '素材尺寸', '广告行业id', '商品类型', '商品id', '广告账户id', '投放时段', '人群定向', '出价']

In [23]:
test_sample = pd.read_csv(root/'test_sample.dat', sep='\t', header=None, names=col_names, iterator=True)
test_sample.get_chunk(5)

,样本id,广告id,创建时间,素材尺寸,广告行业id,商品类型,商品id,广告账户id,投放时段,人群定向,出价
0,1,394352,1529648412,34,84,13,29663,26657,"281474976645120,281474976645120,28147497664512...","age:819,608,988,741,202,837,400,394,942,361,72...",120
1,2,585401,1553076190,40,221,1,-1,6262,"281474976579587,281474976579587,28147497657958...","age:819,433,479,741,229,347,522,79,753,601|edu...",42
2,3,419408,1553031394,30,122,13,32110,17436,"17592185782272,17592185782272,17592185782272,1...",all,6
3,4,405326,1553238836,64,136,1,-1,22359,"281474976694272,281474976694272,28147497669427...","age:333,1|gender:2|area:11505,1874,3790,4566,5...",181
4,5,578942,1541191585,34,12,13,6372,24082,"68719214592,68719214592,68719214592,6871921459...","age:819,608,988,741,202,837,400,394,942,361,72...",31


In [25]:
data = test_sample.get_chunk(50)

In [26]:
data.iloc[1, 9]  # 人群定向

'age:819,608,988,741,202,400,394,942,753,601,433,217,638,1,366,479,229,393,347,787,731,522,333,739'

In [27]:
data.iloc[1, 8]  # 投放时段

'281474976710655,281474976710655,281474976710655,281474976710655,281474976710655,281474976710655,281474976710655'

In [28]:
data['创建时间'] = pd.to_datetime(data['创建时间'], unit='s')

In [30]:
data.head(7)

,样本id,广告id,创建时间,素材尺寸,广告行业id,商品类型,商品id,广告账户id,投放时段,人群定向,出价
55,56,413437,2017-11-07 06:48:13,34,25,1,-1,20851,"281474976710655,281474976710655,28147497671065...",all,70
56,57,497814,2019-03-21 06:14:06,64,224,1,-1,13643,"281474976710655,281474976710655,28147497671065...","age:819,608,988,741,202,400,394,942,753,601,43...",180
57,58,23673,2019-03-13 01:15:54,64,122,1,-1,9563,"281474976710655,281474976710655,28147497671065...","age:819,608,988,741,202,400,394,942,753,601,43...",96
58,59,132394,2016-12-08 01:46:35,34,25,1,-1,2067,"281474976710655,281474976710655,28147497671065...",all,13
59,60,342047,2018-09-06 10:11:38,1,221,13,8786,6283,"274877644800,274877644800,274877644800,2748776...","age:819,608,988,741,202,837,400,394,942,361,72...",25
60,61,702917,2019-03-23 10:33:25,1,21,1,-1,10041,"281474976710655,281474976710655,28147497671065...","area:4063,12347,11253,1939,3293,9452,628,9841,...",74
61,62,425254,2019-03-12 07:36:42,64,21,1,-1,19109,"17592185978880,17592185978880,17592185978880,1...","area:9686,9164",150


- 分块读取文件(整体读取) chunksize
- 迭代读取，测试数据使用 iterator

In [27]:
test_sample1 = pd.read_csv(root/'test_sample.dat', sep='\t', header=None, names=col_names, chunksize=8192)

In [28]:
for chunk in test_sample1:
    print(chunk.shape)

(8192, 11)
(8192, 11)
(3906, 11)
